In [1]:
import json
import pandas as pd
import numpy as np
import math

In [2]:
with open("../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)

with open("../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)

with open("../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)

with open("../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)

with open("../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)

with open("../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [3]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)
out_of_possession_df = all_events_df[all_events_df['eventName'].isin(["Duel", "Foul", "Goalkeeper leaving line", "Others on the ball"])]
passes_df = all_events_df[all_events_df.eventName == "Pass"]

In [27]:
# Determine the previous team ID in the same match
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)

# Identify transitions where the team ID changes and it's within the same match
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Create a transitions DataFrame with required columns for merging
transitions['endSec'] = transitions['eventSec'] + 10
transition_times = transitions[['id', 'matchId', 'eventSec', 'endSec']]

<ipython-input-27-6110caba3bd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
<ipython-input-27-6110caba3bd7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['endSec'] = transitions['eventSec'] + 10


In [30]:
# Convert data types to more memory efficient types
transitions['matchId'] = transitions['matchId'].astype('int32')
transitions['eventSec'] = transitions['eventSec'].astype('float32')
transitions['endSec'] = transitions['endSec'].astype('float32')

# Check if converting teamId to a smaller integer type is possible
transitions['teamId'] = pd.to_numeric(transitions['teamId'], downcast='integer')

<ipython-input-30-cec542fbaf3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['matchId'] = transitions['matchId'].astype('int32')
<ipython-input-30-cec542fbaf3e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['eventSec'] = transitions['eventSec'].astype('float32')
<ipython-input-30-cec542fbaf3e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [51]:
passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
transitions = passes_df[(passes_df['teamId'] != passes_df['previous_teamId']) & (passes_df['previous_teamId'].notna())]

# Add a 10-second window for the transition end time
transitions['transition_end_time'] = transitions['eventSec'] + 10

# Prepare the minimal necessary data for merging
transitions_minimal = transitions[['id', 'matchId', 'eventSec', 'transition_end_time']]
out_of_possession_df_minimal = out_of_possession_df[['id', 'matchId', 'eventSec', 'teamId', "subEventName", "positions", "eventName"]]

<ipython-input-51-f4495ba4cd76>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['previous_teamId'] = passes_df.groupby('matchId')['teamId'].shift(1)
<ipython-input-51-f4495ba4cd76>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transitions['transition_end_time'] = transitions['eventSec'] + 10


In [53]:
# Define a function to process chunks
def process_chunk(chunk):
    chunk_merged = chunk.merge(transitions_minimal, on='matchId', suffixes=('', '_trans'))
    return chunk_merged[(chunk_merged['eventSec'] >= chunk_merged['eventSec_trans']) & (chunk_merged['eventSec'] <= chunk_merged['transition_end_time'])]

# Process in chunks
chunk_size = 100000
result_frames = []
for start in range(0, out_of_possession_df_minimal.shape[0], chunk_size):
    chunk = out_of_possession_df_minimal.iloc[start:start + chunk_size]
    result_frames.append(process_chunk(chunk))

out_of_poss_merged_df = pd.concat(result_frames)
out_of_poss_merged_df = out_of_poss_merged_df.drop_duplicates(subset='id', keep='first')

# out_of_poss_merged_df is ready (update below to create final_df)

In [67]:
out_of_possession_df = out_of_poss_merged_df
out_of_possession_df['uniqueTeamId'] = out_of_possession_df['teamId'].astype(str) + "---" + out_of_possession_df['matchId'].astype(str)

In [69]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x / 33.333) + 1
        if zone == 4:
            zone = 3
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
out_of_possession_df['Zone'] = out_of_possession_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

In [70]:
acc_df = out_of_possession_df[out_of_possession_df.subEventName == "Acceleration"]

In [71]:
def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])
    
acc_df[['calculated_acceleration_length', 'calculated_acceleration_angle']] = acc_df.apply(calculate_pass_details_from_single_column, axis=1)

c:\users\eged1\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [72]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Run"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Run"
    # Side Pass: Otherwise
    else:
        return "Side Run"

acc_df['acceleration_direction'] = acc_df['calculated_acceleration_angle'].apply(determine_pass_direction)

<ipython-input-72-6bec0738f8c8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_df['acceleration_direction'] = acc_df['calculated_acceleration_angle'].apply(determine_pass_direction)


In [73]:
acc_df['zone'] = 'Zone_' + acc_df['Zone'].astype(str) + "_Run"

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
zone_count = acc_df.groupby(['uniqueTeamId', 'zone']).size()

q1, q2, q3 = acc_df['calculated_acceleration_length'].quantile([0.33, 0.66, 1.0])
acc_df['acceleration_length_type'] = pd.cut(acc_df['calculated_acceleration_length'], bins=[0, q1, q2, q3], labels=['Short Run', 'Middle Run', 'Long Run'], include_lowest=True)
acceleration_length_stats = acc_df.groupby(['uniqueTeamId', 'acceleration_length_type']).size().unstack(fill_value=0)
average_acceleration_length = acc_df.groupby('uniqueTeamId')['calculated_acceleration_length'].mean()


# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
zone_count_pivot = zone_count.unstack(fill_value=0)

direction_count = acc_df.groupby('uniqueTeamId')['acceleration_direction'].value_counts().unstack(fill_value=0)

# Add the number of forward runs in Zone 3
acc_df['is_forward_run_zone_3'] = ((acc_df['acceleration_direction'] == 'Forward Run') & (acc_df['Zone'] == 3))
forward_runs_zone_3 = acc_df.groupby('uniqueTeamId')['is_forward_run_zone_3'].sum()

# Concatenate DataFrames
acceleration_direction_df = pd.concat([zone_count_pivot, direction_count, acceleration_length_stats, average_acceleration_length.rename('average_acceleration_length'), forward_runs_zone_3.rename('Forward_Runs_Zone_3')], axis=1)

<ipython-input-73-32ca132416b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_df['zone'] = 'Zone_' + acc_df['Zone'].astype(str) + "_Run"
<ipython-input-73-32ca132416b8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_df['acceleration_length_type'] = pd.cut(acc_df['calculated_acceleration_length'], bins=[0, q1, q2, q3], labels=['Short Run', 'Middle Run', 'Long Run'], include_lowest=True)
<ipython-input-73-32ca132416b8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [74]:
filtered_df = out_of_possession_df[
    (out_of_possession_df['subEventName'] == 'Foul') |
    (out_of_possession_df['subEventName'] == 'Touch') |
    (out_of_possession_df['subEventName'].str.contains('duel', case=False, na=False))
]

filtered_df['duel_zone'] = filtered_df['subEventName'].astype(str) + '_Zone_' + filtered_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
duel_zone_count = filtered_df.groupby(['uniqueTeamId', 'duel_zone']).size().unstack(fill_value=0)

all_count = out_of_possession_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)

grouped_out_of_poss_df = pd.concat([duel_zone_count, all_count], axis=1)

<ipython-input-74-25869d8d3c46>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['duel_zone'] = filtered_df['subEventName'].astype(str) + '_Zone_' + filtered_df['Zone'].astype(str)


In [75]:
out_of_final_df = pd.concat([grouped_out_of_poss_df, acceleration_direction_df], axis=1)
out_of_final_df.isna().sum()

Air duel_Zone_1                    0
Air duel_Zone_2                    0
Air duel_Zone_3                    0
Foul_Zone_1                        0
Foul_Zone_2                        0
Foul_Zone_3                        0
Ground attacking duel_Zone_1       0
Ground attacking duel_Zone_2       0
Ground attacking duel_Zone_3       0
Ground defending duel_Zone_1       0
Ground defending duel_Zone_2       0
Ground defending duel_Zone_3       0
Ground loose ball duel_Zone_1      0
Ground loose ball duel_Zone_2      0
Ground loose ball duel_Zone_3      0
Touch_Zone_1                       0
Touch_Zone_2                       0
Touch_Zone_3                       0
Acceleration                       0
Air duel                           0
Clearance                          0
Foul                               0
Goalkeeper leaving line            0
Ground attacking duel              0
Ground defending duel              0
Ground loose ball duel             0
Hand foul                          0
L

In [76]:
# if acceleration is 0, all acceleration related cols are nan. let's make them 0
out_of_final_df.fillna(0, inplace = True)

out_of_final_df.to_csv('final_transition_out_of_poss_df.csv', index=True)

# END OF FILE